# Using Masks In Pygame 

Masks are the third element of the visuals which brings us much more control over the visual by calculating things in a pixelian way ( which we do it by rectangles default) 

<img src="img/intro2Masks.jpg" width="600px">

For an example we normally control collusions by rectangle . But with the help of the masks , we can detect collusions by pixels. 

### How mask works ? 

<img src="img/turning_image_surface_into_masks.jpg" width="600">

### What can we do with masks ??? 

<img src="img/what_do_we_use_masks_for.jpg" width="600px">

There should be two thing which we should remember while using masks with sprites 

> Having an animation for sprite requires changing surface of that character. Because of that in every frame of the animation you have to mask it again and again and again ... 

> Calculating if the two pixels are touching is the way more harder than calculating if the rectangles are colliding each other . Because of that I recommend you to check if there is rectangle collusion first . Than check if there is a mask collusion (for optimization)

#### Implementation Of Masks 1 - Return to another color when there is a pixel collusion

In [1]:
import pygame  
import random 


WIDTH , HEIGHT = 800 , 800
pygame.init() 
screen = pygame.display.set_mode((WIDTH , HEIGHT))  
clock = pygame.time.Clock()

class CursorBox(pygame.sprite.Sprite): 
    def __init__(self, color , size = 50) -> None:
        super().__init__() 
        self.color = color 
        self.size = size 

        self.image = pygame.Surface([self.size , self.size]) 
        self.rect = self.image.get_rect(center = (300,300))  
        self.image.fill(self.color) 
        self.mask = pygame.mask.from_surface(self.image) 

    def update(self): 
        if pygame.mouse.get_pos(): 
            self.rect.center = pygame.mouse.get_pos() 
            self.image.fill(self.color)  

    def changeColor(self): 
        self.color = (random.randint(40,200),random.randint(40,200),random.randint(40,200))

class StaticAlpha(pygame.sprite.Sprite): 

    def __init__(self) -> None:
        super().__init__() 
        self.image = pygame.image.load("img/alpha.png").convert_alpha()  
        self.w , self.h = self.image.get_width() , self.image.get_height()
        self.rect = self.image.get_rect(center = ((WIDTH)//2 , (HEIGHT)//2))  


player = pygame.sprite.GroupSingle(CursorBox((255,0,0))) 
alpha = pygame.sprite.GroupSingle(StaticAlpha()) 

running = True 
while running : 
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT: 
            exit() 
            pygame.quit() 


    if pygame.sprite.spritecollide(player.sprite , alpha , False): # this is for optimization 
        # Because calculating every pixel every times slows down your game very much 
        # Also worth mentioning that the sprite.spritecollide() method checks for rectangle collusion by default unless you dont change the 
        # ... third parameter as pygame.sprite.collide_mask 
        if pygame.sprite.spritecollide(player.sprite , alpha , False , pygame.sprite.collide_mask): 
            player.sprite.changeColor() 

    screen.fill((255 , 255 , 204))
    player.draw(screen) 
    alpha.draw(screen) 
    player.update() 
    pygame.display.update()  
    clock.tick(60) 

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

#### Implementation of Masks 2 - Creating masks without the sprite class 

> Note : Overlap function needs offsets . What an offset represents is given in the image down bellow : 

<img src="img/offsets_for_masks.jpg" width="600px">  

> O and P are the masks that overlaps with eachother , the length of the x and y lines represents the magnitude of offsets  

In [1]:

import pygame 


pygame.init() 
WIDTH , HEIGHT = 800 , 800  
SQUARE_SIZE = 50 

RED = (255,0,0) 
WHITE = (255,255,255)
screen = pygame.display.set_mode((WIDTH , HEIGHT))  

player_surface = pygame.Surface([SQUARE_SIZE , SQUARE_SIZE]) 
player_rect = player_surface.get_rect(center = (300,300)) 
player_surface.fill(RED)   
player_mask = pygame.mask.from_surface(player_surface) 

alpha = pygame.image.load("img/alpha.png").convert_alpha()  
alpha_rect = alpha.get_rect(center = (WIDTH//2 , HEIGHT//2)) 
alpha_mask = pygame.mask.from_surface(alpha)  
alpha_pos = alpha_rect.topleft
 

clock = pygame.time.Clock()
running = True 

while running: 

    rect_color = (255,0,0) 
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:
            running = False 
            exit() 
            pygame.quit() 

    
    if player_rect.colliderect(alpha_rect): 
        
        offset_x  = alpha_pos[0] - player_rect.left # if alpha_pos = 100,100 , and player.rect.left = 300 then this returns 100-300 = *_200_*
        offset_y = alpha_pos[1] - player_rect.top 
        #if player_mask.overlap(alpha_mask ,(offset_x , offset_y)):  
            #rect_color = (0,255,0) 
            ## Very sensetive overlap mechanic that triggers even 1 pixel is overlapping 
            ## player_mask.overlap() returns the pixel cordinates that overlaps with the border of the other mask 

        OVERLAP_TOLERANCE = 200 # Tolarate maximum 200 overlapping pixels . If there is more , trigger the statement
        if player_mask.overlap_area(alpha_mask ,(offset_x , offset_y) ) > OVERLAP_TOLERANCE:  
            rect_color = (0,0,255)  

        # Third type of detecting overlap between masks is `overlap_mask()` method which returns the pixels that overlaps between 2 masks as a mask 
        # With the help of that you can change the color of which parts are touching each other 
        



            
    screen.fill(WHITE) 
    screen.blit(alpha , alpha_rect)  
    player_rect.center = pygame.mouse.get_pos() 
    player_surface.fill(rect_color)  

    screen.blit(player_surface , player_rect) 
    
    pygame.display.update() 
    clock.tick(60)


pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

### Implementation 3 - Creating masks to fill a Surface 

Filling a surface can be done  with the masks by following  these stages : 

<img src="img/filling_a_surface_with_masks.jpg" width = "600px">

If you ask why we need these complex stages to filling a surface is , surfaces have large varies of colored pixels , this makes it hard to identify which part of the surface to fill the color. To get away from this problem , we need masks that turns the surface in a simpe way. 

<img src="img/why_we_cant_fill_surfaces_without_masks.jpg" width="600px">

In [1]:
import pygame 


pygame.init() 
WIDTH , HEIGHT = 800 , 800  
SQUARE_SIZE = 50 

RED = (255,0,0) 
WHITE = (255,255,255) 
BLACK = (0,0,0) 
PURPLE = (204 , 30 , 102)
screen = pygame.display.set_mode((WIDTH , HEIGHT))  


alpha = pygame.image.load("img/alpha.png").convert_alpha()  
alpha_rect = alpha.get_rect(center = (WIDTH//2 , HEIGHT//2)) 
alpha_mask = pygame.mask.from_surface(alpha)  
alpha_pos = alpha_rect.topleft 

# Creating new surface : 
alpha_new_surf = alpha_mask.to_surface() 
alpha_new_surf.set_colorkey(BLACK) # makes the Black pixels of the surface unvisible (but they still exist) 

surf_w , surf_h = alpha_new_surf.get_size() 
for x in range(surf_w): 
    for y in range(surf_h): 
        if alpha_new_surf.get_at((x,y))[0] != 0 :  #alpha_new_surf.get_at((x,y)) returns (r,g,b) 
            alpha_new_surf.set_at((x,y) , (30,102,204)) 

 

clock = pygame.time.Clock()
running = True 

while running: 

    rect_color = (255,0,0) 
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:
            running = False 
            exit() 
            pygame.quit() 
    screen.fill(PURPLE) 
    screen.blit(alpha , alpha_rect)   
    screen.blit(alpha_new_surf , alpha_rect)# masking workflow : if the pixel is colored return 1 else return 0 
                                            #  masking to surface workflow : if the value is 1 return WHITE elif the value is 0 return BLACK 
    
    pygame.display.update() 
    clock.tick(60)


pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

### Implementation 4 - Creating Outlines by Using Masks

There are two ways to make outlines , first way is the easiest one but it is inaccurate . The other way is more complex but accurate .  

<img src="img/creating_an_outline.jpg" width="600px">

> Easiest one is the `mask.outline()` method which can detect the outlines in the out of the surface . But it cant detect the outlines thats inside of the surface (like alpha icon has a hole at the center .  This method cant outline the hole part)

In [1]:
import pygame 


pygame.init() 
WIDTH , HEIGHT = 800 , 800  
SQUARE_SIZE = 50 

RED = (255,0,0) 
WHITE = (255,255,255) 
BLACK = (0,0,0) 
PURPLE = (204 , 30 , 102) 
LIGHT_BLUE = (30 , 102 , 204)
screen = pygame.display.set_mode((WIDTH , HEIGHT))  


alpha = pygame.image.load("img/alpha.png").convert_alpha()  
alpha_rect = alpha.get_rect(center = (WIDTH//2 , HEIGHT//2)) 
alpha_mask = pygame.mask.from_surface(alpha)   
alpha_pos = alpha_rect.topleft



clock = pygame.time.Clock()
running = True 

while running: 

    rect_color = (255,0,0) 
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:
            running = False 
            exit() 
            pygame.quit() 
    screen.fill(WHITE)  
    screen.blit(alpha , alpha_rect)  

    for pixel_cord in alpha_mask.outline(): 
        pix_x , pix_y = pixel_cord  
        off_x , off_y = alpha_pos
        pygame.draw.circle(screen , LIGHT_BLUE , (pix_x + off_x , pix_y + off_y) , 5) 

      
    
    pygame.display.update() 
    clock.tick(60)

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

The more harder , more complex way to outline is blitting the surface with different offsets some times . 

In [1]:
import pygame 


pygame.init() 
WIDTH , HEIGHT = 800 , 800  
SQUARE_SIZE = 50 

RED = (255,0,0) 
WHITE = (255,255,255) 
BLACK = (0,0,0) 
PURPLE = (204 , 30 , 102)
screen = pygame.display.set_mode((WIDTH , HEIGHT))  


alpha = pygame.image.load("img/alpha.png").convert_alpha()  
alpha_rect = alpha.get_rect(center = (WIDTH//2 , HEIGHT//2)) 
alpha_mask = pygame.mask.from_surface(alpha)  
alpha_pos = alpha_rect.topleft 

# Creating new surface : 
alpha_new_surf = alpha_mask.to_surface() 
alpha_new_surf.set_colorkey(BLACK) # makes the Black pixels of the surface unvisible (but they still exist) 

surf_w , surf_h = alpha_new_surf.get_size() 
for x in range(surf_w): 
    for y in range(surf_h): 
        if alpha_new_surf.get_at((x,y))[0] != 0 :  #alpha_new_surf.get_at((x,y)) returns (r,g,b) 
            alpha_new_surf.set_at((x,y) , (30,102,204)) 

 

clock = pygame.time.Clock()
running = True 

while running: 

    rect_color = (255,0,0) 
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:
            running = False 
            exit() 
            pygame.quit() 
    screen.fill(PURPLE)  


    offset_outline = 10
    for _ in range(2): 
        for _ in range(4):  
            x , y = alpha_pos
            screen.blit(alpha_new_surf , (x + offset_outline , y)) 
            screen.blit(alpha_new_surf , (x  , y+ offset_outline))  
            screen.blit(alpha_new_surf , (x + offset_outline , y+ offset_outline))  
            screen.blit(alpha_new_surf , (x - offset_outline , y+ offset_outline))  

        offset_outline *= -1
    screen.blit(alpha , alpha_rect)   
     
    
    pygame.display.update() 
    clock.tick(60)

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

### Implementation 4 : Coloring overlapping Areas with Masks

In [1]:
import pygame 


pygame.init() 
WIDTH , HEIGHT = 800 , 800  
SQUARE_SIZE = 50 

RED = (255,0,0) 
WHITE = (255,255,255) 
BLACK = (0,0,0) 
PURPLE = (204 , 30 , 102) 
screen = pygame.display.set_mode((WIDTH , HEIGHT))  


alpha = pygame.image.load("img/alpha.png").convert_alpha()  
alpha_rect = alpha.get_rect(center = (WIDTH//2 , HEIGHT//2)) 
alpha_mask = pygame.mask.from_surface(alpha)  
alpha_pos = alpha_rect.topleft 
 
player = pygame.image.load("img/ship.png").convert_alpha()
player_rect = player.get_rect(center = (100,100)) 
player_mask = pygame.mask.from_surface(player) 
player_pos = player_rect.topleft



 

clock = pygame.time.Clock()
running = True 

while running: 

    rect_color = (255,0,0) 
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:
            running = False 
            exit() 
            pygame.quit() 
    screen.fill(WHITE) 
    screen.blit(alpha , alpha_rect)  
    
    if pygame.mouse.get_pos():
        player_rect.center = pygame.mouse.get_pos()  
        screen.blit(player , player_rect)

    offset_x = alpha_pos[0] - player_rect.left 
    offset_y = alpha_pos[1] - player_rect.top 
    if player_mask.overlap(alpha_mask , (offset_x , offset_y)): 
        new_mask = player_mask.overlap_mask(alpha_mask , (offset_x , offset_y)) 
        new_surf = new_mask.to_surface() 
        new_surf.set_colorkey(BLACK)  

        surf_w , surf_h = new_surf.get_size() 
        for x in range(surf_w): 
            for y in range(surf_h): 
                if new_surf.get_at((x,y))[0] != 0 : 
                    new_surf.set_at((x,y) , 'red')   
        screen.blit(new_surf , player_rect) 

    
    
    pygame.display.update() 
    clock.tick(60)

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 